# Mastering Machine Learning on AWS
## Section 2, Chapter 2
### Classification and the Naive Bayes Algorithm

In the book the authors use Twitter data.  However, in the X API a paid subscription is required to pull posts from the example threads.  Thus, in this notebook we will pull posts from Reddit, examining the text located in the body of these posts.

# Contents

1. [Weather Data Example](#Weather-Data-Example)
1. [Creating the text dataset](#Creating-the-text-dataset)  
    1. [Pulling data from Reddit for the political example](#Pulling-data-from-Reddit-for-the-political-example)
        1. [Republicans Data](#Republicans-Data)
        1. [Democrats Data](#Democrats-Data)  
    1. [Create the combined text dataframe](#Create-the-combined-text-dataframe)
    1. [Feature Transformations](#Preparing-the-Data-with-Feature-Transformations)
        1. [Stop Words](#Stop-Words)
        1. [Feature Function](#Feature-Function)
        1. [Score Function](#Score-Function)
    1. [Naive Bayes Algorithm](#Naive-Bayes-algorithm)  

In [1]:
import sys
sys.executable

'/Users/blakewallace/anaconda3/envs/linear_regression_env/bin/python'

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mlp
import matplotlib.pyplot as plt

import praw   # Python Reddit API Wrapper
import datetime as dt
import time
import requests

import nltk
from nltk.stem import WordNetLemmatizer
# nltk.download('stopwords') 
# nltk.download('punkt_tab')
# nltk.download("wordnet")
# nltk.download("omw-1.4")

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix

%matplotlib inline

class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

import bnaivebayes
# import FeatureFunction
import creds

In [5]:
!/Users/blakewallace/anaconda3/envs/linear_regression_env/bin/python -m pip install update kagglehub

In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dipankarsrirag/topic-modelling-on-emails")

print("Path to dataset files:", path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10.4M/10.4M [00:06<00:00, 1.59MB/s]

Extracting files...


Path to dataset files: /Users/blakewallace/.cache/kagglehub/datasets/dipankarsrirag/topic-modelling-on-emails/versions/1


In [12]:
with open('/Users/blakewallace/.cache/kagglehub/datasets/dipankarsrirag/topic-modelling-on-emails/versions/1/Data/Crime/15390.txt', "r") as document:
    for line in document:
        print(line)



:    Indeed, if NSA really designed the algorithm to be secure, it's very likely

: as secure as IDEA or 2-key DES.  However, the system as a whole isn't resistant

: to "practical cryptanalysis."  In _The Puzzle Palace_, Bamford describes how

: several NSA employees were turned by foreign (presumably KGB) agents, despite

: security measures that I doubt any Big 8 accounting firm could match.  And

: NSA confidential data was *not* subject to being requested by thousands of

: police organizations and courts across the land.



Ah yes, don't anyone mention Ronald William Pelton[*], heh heh heh.  How

embarrassing.



G

[*: NSA, 1964-1979; KGB 1980-1985]



In [2]:
import os

def read_text_files(folder_path):
    """
    Reads all text files in a specified folder and returns their content.

    Args:
        folder_path (str): The path to the folder containing the text files.

    Returns:
        dict: A dictionary where keys are file names and values are file contents.
               Returns an empty dictionary if the folder does not exist or 
               no text files are found.
    """
    file_contents = {}
    if not os.path.exists(folder_path):
        print(f"Error: Folder '{folder_path}' not found.")
        return file_contents
    
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            try:
                with open(file_path, 'r') as file:
                    file_contents[filename] = file.read()
            except Exception as e:
                 print(f"Error reading {filename}: {e}")
    return file_contents

# Example Usage
folder_path = "/Users/blakewallace/.cache/kagglehub/datasets/dipankarsrirag/topic-modelling-on-emails/versions/1/Data/Crime/"  # Replace with the actual path to your folder
text_files_data = read_text_files(folder_path)

if text_files_data:
    for filename, content in text_files_data.items():
        print(f"Contents of {filename}:\n{content}\n---")
else:
    print("No text files found or an error occurred.")

Error reading 15672.txt: 'utf-8' codec can't decode byte 0xa0 in position 896: invalid start byte
Contents of 14147.txt:

Archive-name: ripem/faq
Last-update: Sun, 7 Mar 93 21:00:00 -0500

ABOUT THIS POSTING
------------------
This is a (still rather rough) listing of likely questions and
information about RIPEM, a program for public key mail encryption.  It
(this FAQ, not RIPEM) was written and will be maintained by Marc
VanHeyningen, <mvanheyn@whale.cs.indiana.edu>.  It will be posted to a
variety of newsgroups on a monthly basis; follow-up discussion specific
to RIPEM is redirected to the group alt.security.ripem.

This month, I have reformatted this posting in an attempt to comply
with the standards for HyperText FAQ formatting to allow easy
manipulation of this document over the World Wide Web.  Let me know
what you think.

DISCLAIMER
----------
Nothing in this FAQ should be considered legal advice, or anything
other than one person's opinion.  If you want real legal advice, talk


In [3]:
len(text_files_data)

1099

In [13]:
text_files_data[list(text_files_data.keys())[0]]

'\nArchive-name: ripem/faq\nLast-update: Sun, 7 Mar 93 21:00:00 -0500\n\nABOUT THIS POSTING\n------------------\nThis is a (still rather rough) listing of likely questions and\ninformation about RIPEM, a program for public key mail encryption.  It\n(this FAQ, not RIPEM) was written and will be maintained by Marc\nVanHeyningen, <mvanheyn@whale.cs.indiana.edu>.  It will be posted to a\nvariety of newsgroups on a monthly basis; follow-up discussion specific\nto RIPEM is redirected to the group alt.security.ripem.\n\nThis month, I have reformatted this posting in an attempt to comply\nwith the standards for HyperText FAQ formatting to allow easy\nmanipulation of this document over the World Wide Web.  Let me know\nwhat you think.\n\nDISCLAIMER\n----------\nNothing in this FAQ should be considered legal advice, or anything\nother than one person\'s opinion.  If you want real legal advice, talk\nto a real lawyer.\n\nQUESTIONS AND ANSWERS\n---------------------\n\n1)  What is RIPEM?\n\n RIPEM

In [14]:
bnaivebayes.FeatureFunction(text_files_data[list(text_files_data.keys())[0]])

[('archivename', 1),
 ('ripemfaq', 1),
 ('lastupdate', 1),
 ('sun', 1),
 ('mar', 1),
 ('post', 5),
 ('still', 1),
 ('rather', 2),
 ('rough', 1),
 ('list', 1),
 ('likely', 1),
 ('question', 2),
 ('information', 2),
 ('ripem', 38),
 ('program', 4),
 ('public', 13),
 ('key', 34),
 ('mail', 10),
 ('encryption', 4),
 ('faq', 3),
 ('write', 4),
 ('maintain', 1),
 ('marc', 1),
 ('vanheyningen', 1),
 ('mvanheynwhalecsindianaedu', 1),
 ('variety', 1),
 ('newsgroups', 1),
 ('monthly', 1),
 ('basis', 1),
 ('followup', 1),
 ('discussion', 1),
 ('specific', 1),
 ('redirect', 1),
 ('group', 1),
 ('altsecurityripem', 2),
 ('month', 1),
 ('reformatted', 1),
 ('attempt', 3),
 ('comply', 1),
 ('standards', 3),
 ('hypertext', 1),
 ('format', 1),
 ('allow', 7),
 ('easy', 4),
 ('manipulation', 1),
 ('document', 4),
 ('world', 1),
 ('wide', 1),
 ('web', 1),
 ('let', 1),
 ('know', 3),
 ('think', 2),
 ('disclaimer', 2),
 ('nothing', 1),
 ('consider', 4),
 ('legal', 2),
 ('advice', 2),
 ('anything', 2),
 ('one

In [ ]:
bnaivebayes.

In [70]:
import json

reddit_creds = {
    'client_id': 'OnvJDrnt8doq4w',
    'client_secret': '0xQvVRkHk5RnLNATJMFXK5EPTFk',
    'user_agent': 'Blake Wallace',
    'username': 'Bw1097246',
    'password': 'Bw00436948$'
}
myJSON = json.dumps(reddit_creds)

with open("reddit_creds.json", "w") as jsonfile:
    jsonfile.write(myJSON)
    print("Write successful")

Write successful


In [71]:
aws_creds = {
    'account_id_root': 324037306813,
    'iam_username_root': 'BlakeWallace',
    'password_root': "Bw$0196360$root",
    'account_id': 324037306813,
    'user_name': 'BlakeAdmin',
    'password': 'Bw$0196360$admin'
}
myJSON = json.dumps(aws_creds)

with open("aws_creds.json", "w") as jsonfile:
    jsonfile.write(myJSON)
    print("Write successful")

Write successful


In [63]:
import json

article_info = {
    "domain" : "tutswiki",
    "language" : "python",
    "date" : "11/09/2020",
    "topic" : "config file"
}
myJSON = json.dumps(article_info)

with open("tutswiki.json", "w") as jsonfile:
    jsonfile.write(myJSON)
    print("Write successful")

Write successful


In [2]:
import json

with open("./creds/reddit_creds.json", "r") as jsonfile:
    reddit_creds = json.load(jsonfile)
    print("Read successful")
print(reddit_creds)

Read successful
{'client_id': 'OnvJDrnt8doq4w', 'client_secret': '0xQvVRkHk5RnLNATJMFXK5EPTFk', 'user_agent': 'Blake Wallace', 'username': 'Bw1097246', 'password': 'Bw00436948$'}


In [68]:
import json

article_info = {
    "domain" : "tutswiki",
    "language" : "python",
    "date" : "11/09/2020",
    "topic" : "config file"
}

with open("tutswiki.json", "r") as jsonfile:
    data = json.load(jsonfile) # Reading the file
    print("Read successful")
    jsonfile.close()
    
data['date'] = '12/09/2020' # Updating, before it was 11/09/2020
print("Date updated from 11/09/2020 to 12/09/2020")
with open("tutswiki.json", "w") as jsonfile:
    myJSON = json.dump(data, jsonfile) # Writing to the file
    print("Write successful")
    jsonfile.close()

Read successful
Date updated from 11/09/2020 to 12/09/2020
Write successful


In [69]:
import json

with open("tutswiki.json", "r") as jsonfile:
    data = json.load(jsonfile)
    print("Read successful")
print(data)

Read successful
{'domain': 'tutswiki', 'language': 'python', 'date': '12/09/2020', 'topic': 'config file'}


## Weather Data Example

In [2]:
weather_data = pd.DataFrame(
    {'Temperature (in F)': ['Less than 20', '20-32', '32-70', '70 and above', '20-32', '32-70', 
                            'Less than 20', '32-70', '20-32', 'Less than 20'], 
     'Sky condition': ['Sunny', 'Sunny', 'Cloudy', 'Cloudy', 'Cloudy', 'Sunny', 'Cloudy', 'Sunny', 'Cloudy', 'Sunny'], 
     'Wind speed (in MPH)': [30, 6, 20, 0, 10, 15, 8, 7, 11, 13], 
     'Snowfall': [False, False, False, False, True, False, True, False, False, True]})



In [3]:
weather_data

,Temperature (in F),Sky condition,Wind speed (in MPH),Snowfall
0,Less than 20,Sunny,30,False
1,20-32,Sunny,6,False
2,32-70,Cloudy,20,False
3,70 and above,Cloudy,0,False
4,20-32,Cloudy,10,True
5,32-70,Sunny,15,False
6,Less than 20,Cloudy,8,True
7,32-70,Sunny,7,False
8,20-32,Cloudy,11,False
9,Less than 20,Sunny,13,True


`Sky condition` - Nominal or categorical, values: **Sunny** or **Cloudy**.  We can one-hot encode this feature.  
`Temperature (in F)` - Ordinal, values: **Less than 20**, **20-32**, **32-70**, **70 and above**.  Interpolate the range of values to a defined scale.  
`Wind Speed (in MPH)` - Continuous, values range: **0-30**.  Note, in some algorithms we would need to discretize this variable during data preprocessing before training ML algorithms.  
`Snowfall` - Nominal or Boolean, values: **True** or **False**.  Convert to "1" and "0" before processing in some ML algorithms.  

The Naive Bayes algorithm is compariable to how a belief system evolves.  

In [4]:
weather_data.Snowfall.value_counts(normalize=True)

Snowfall
False    0.7
True     0.3
Name: proportion, dtype: float64

In [5]:
# Likelihood in Example, `Temperature (in F)` is "20-32," when it Snows
likelihood = weather_data[weather_data.Snowfall == True]['Temperature (in F)'].value_counts(normalize=True)['20-32']
likelihood

np.float64(0.3333333333333333)

In [6]:
# Prior, likelihood of it snowing
prior = weather_data['Snowfall'].value_counts(normalize=True)[True]
prior

np.float64(0.3)

In [7]:
# Evidence, probability of the Temperature being "20-32"
evidence = weather_data['Temperature (in F)'].value_counts(normalize=True)['20-32']
evidence

np.float64(0.3)

In [8]:
# Posterior, probability that it Snows given the Temperature being "20-32"
posterior = prior*likelihood/ evidence
print(str(posterior * 100) + '%')

33.33333333333333%


In [9]:
# Same posterior can be calculated
print(str(weather_data[weather_data['Temperature (in F)'] == '20-32'].Snowfall.value_counts(normalize=True)[True]*100) + "%")

33.33333333333333%


## Creating the text dataset
for creating a Naive Bayes Classifier
### Pulling data from Reddit for the political example

There are two requests being made, one for democratic labelled data and one for republican labelled data.

#### Republicans Data

In [5]:
import praw   # Python Reddit API Wrapper
import creds

start_time = time.time()

reddit = praw.Reddit(client_id=creds.CLIENT_ID, \
                     client_secret=creds.CLIENT_SECRET, \
                     user_agent=creds.USER_AGENT, \
                     username=creds.USERNAME, \
                     password=creds.PASSWORD)

In [6]:
print(reddit.user.me())

Bw1097246


In [7]:
# submission = reddit.
submission = reddit.submission('1hfl187')
print(submission.title)  # to make it non-lazy
# print(vars(submission))

Maybe Democrats Didn’t Do So Badly After All


In [6]:
reddit_creds

{'client_id': 'OnvJDrnt8doq4w',
 'client_secret': '0xQvVRkHk5RnLNATJMFXK5EPTFk',
 'user_agent': 'Blake Wallace',
 'username': 'Bw1097246',
 'password': 'Bw00436948$'}

In [3]:
# this is pulling data from the subreddit category for 

####   Republican    ####

start_time = time.time()

reddit = praw.Reddit(client_id=reddit_creds['client_id'], \
                     client_secret=reddit_creds['client_secret'], \
                     user_agent=reddit_creds['user_agent'], \
                     username=reddit_creds['username'], \
                     password=reddit_creds['password'])

SUBREDDIT = 'republicans'
# SUBREDDIT = 'legostarwars'
LIMIT = 2000
# LIMIT = 1

# instantiating a subreddit
# subreddit = reddit.subreddit('legostarwars')
subreddit = reddit.subreddit(SUBREDDIT)

# pulling from the subreddit in the five diffeent categories.  Note the 4 second intervals between each pull from reddit.
top_subreddit = subreddit.top(limit=LIMIT);


time.sleep(4)
hot_subreddit = subreddit.hot(limit=LIMIT);


time.sleep(4)
new_subreddit = subreddit.new(limit=LIMIT);


time.sleep(4)
controversial_subreddit = subreddit.controversial(limit=LIMIT);


# time.sleep(4)
# gilded_subreddit = subreddit.gilded(limit=LIMIT);



lst = ['top_subreddit', 'hot_subreddit', 'new_subreddit', 'controversial_subreddit'] #, 'gilded_subreddit']
# lst = ['top_subreddit', 'hot_subreddit']

data_dict = {
    'title'    : [],
    'score'    : [],
    'id'       : [],
    'url'      : [],
    'comms_num': [],
    'created'  : [],
    'body'     : [],
    'category' : [],
    'class'    : []
}

duplicates_dict = {
    'title'    : [],
    'score'    : [],
    'id'       : [],
    'url'      : [],
    'comms_num': [],
    'created'  : [],
    'body'     : [],
    'category' : [],
    'class'    : []
}


for item in lst:
    # print(f'category for Lego Star Wars: {item}')
    print(f'category for {SUBREDDIT}: {item}')
    for submission in eval(item):
#         print(submission, submission.id)
        if submission.id in set(data_dict['id']):
            duplicates_dict['title'].append(submission.title)
            duplicates_dict['score'].append(submission.score)
            duplicates_dict['id'].append(submission.id)
            duplicates_dict['url'].append(submission.url)
            duplicates_dict['comms_num'].append(submission.num_comments)
            duplicates_dict['created'].append(submission.created)
            duplicates_dict['body'].append(submission.selftext)
            duplicates_dict['category'].append(item)
            duplicates_dict['class'] = 1
        
        else:
#             print(submission)
            data_dict['title'].append(submission.title)
            data_dict['score'].append(submission.score)
            data_dict['id'].append(submission.id)
            data_dict['url'].append(submission.url)
            data_dict['comms_num'].append(submission.num_comments)
            data_dict['created'].append(submission.created)
            data_dict['body'].append(submission.selftext)
            data_dict['category'].append(item)
            data_dict['class'] = 1
#             print(item)
# print(len(data_dict['id']))
            


# constructing a data frame from the data
positive_data = pd.DataFrame(data_dict)


# This function is taken from Rodrigues' website.  It converts the initial time stamp, which is a UNIX timestamp,
# into something we can understand.
def get_date(created):
    return dt.datetime.fromtimestamp(created)


# applying the timestamp function to create a new row.  This is how Rodrigues did it.  We follow his lead.
_timestamp = positive_data['created'].apply(get_date)


# Now, we create a new row in the dataframe to hold the new time stamp information
positive_data = positive_data.assign(timestamp = _timestamp)


print()
print('The totle number of posts pulled from the surver:' + color.BOLD + f' {len(data_dict["id"]) + len(duplicates_dict["id"])}' + color.END)
print()
print('The number of unique posts in the new dictionary:' + color.BOLD + f' {len(data_dict["id"])}' + color.END)
print('The number of posts in the duplicates dictionary:' + color.BOLD + f' {len(duplicates_dict["id"])}' + color.END)
print()
print(f'Our new dataframe is defined as: positive_data')



end_time = round(time.time() - start_time, 3)
print(f'time: {end_time} seconds')
end_time_minutes = int(end_time/ 60)
end_time_seconds = round(end_time % 60, 3)

print(f'time: {end_time_minutes} minutes, {end_time_seconds} seconds')
      
      

category for republicans: top_subreddit
category for republicans: hot_subreddit
category for republicans: new_subreddit
category for republicans: controversial_subreddit

The totle number of posts pulled from the surver: 3923

The number of unique posts in the new dictionary: 2843
The number of posts in the duplicates dictionary: 1080

Our new dataframe is defined as: positive_data
time: 74.253 seconds
time: 1 minutes, 14.253 seconds


In [4]:
Positive_data_df = positive_data[positive_data['body'] != '']['body'].reset_index()
# print('There are' + color.BOLD + f' {Positive_data_df.shape[0]} ' + color.END + 'Posts about Lego Starwars that have content in the body of the posts.')
print('There are' + color.BOLD + f' {Positive_data_df.shape[0]} ' + color.END + 'Republican posts that have content in the body of the posts.')


There are 172 Republican posts that have content in the body of the posts.


In [12]:
#### ONLY RUN THIS CELL IF YOU WANT TO OVERRIDE THE CURRENT DATA, WHICH HAS 200 POSTS

# Positive_data_df = positive_data[positive_data['body'] != '']['body'].reset_index().iloc[:200]
# Positive_data_df.drop(columns=['index'], inplace=True)
# Positive_data_df.to_csv('./data/republicans.csv', index=False)

#### Define the Republican (positive/1) dataframe

In [4]:
pos_data_df_lego = pd.read_csv('~/Desktop/DSI/Projects/Project_3/project_3/Deliverables/data/data_neg_class_initial.csv')

In [26]:
pos_data_df_lego[pos_data_df_lego['body'].notnull()]['body']

2       Hi, I’m Barack Obama, President of the United ...
10      I'm rich and bored. Make me proud Reddit! \n\n...
15      Ellen Pao resigned from reddit today by mutual...
16      **My 6 Questions:**\n\n1. How did you enjoy yo...
26      Hello Reddit,\n\nI’m just a normal guy, who sp...
                              ...                        
6454    The block sucks.  It's sucked since the beginn...
6465    Hey, just your common young teen here. \n\nIf ...
6521           And get member berries all over the place?
6531    Good old days where when “Window” was a square...
6538    As a kid I often found myself holding my poop ...
Name: body, Length: 544, dtype: object

In [2]:
positive_data_df = pd.read_csv('./data/republicans.csv')
positive_data_df

,body
0,"So obviously as the title reads, I am a democr..."
1,Love from Canada 🇨🇦❤️🇺🇸
2,[ Removed by Reddit on account of violating th...
3,Just thought i'd point that out due to the ove...
4,"Remember, a MAGA hat is worth nothing unless i..."
...,...
195,In recent months much has been discussed about...
196,It is my wish that they would have the respect...
197,\nhttps://youtu.be/BfK0ynBtVoQ?si=DKJP4_qaHevU...
198,My biggest problem with voting for Romney is h...


#### Democrats Data

In [2]:
# this is pulling data from the subreddit category for 

####    Democrats    ####

start_time = time.time()


reddit = praw.Reddit(client_id=reddit_creds['client_id'], \
                     client_secret=reddit_creds['client_secret'], \
                     user_agent=reddit_creds['user_agent'], \
                     username=reddit_creds['username'], \
                     password=reddit_creds['password'])

SUBREDDIT = 'democrats'
LIMIT = 1000
# LIMIT = 5000

# instantiating a subreddit
# subreddit = reddit.subreddit('legostarwars')
subreddit = reddit.subreddit(SUBREDDIT)

# pulling from the subreddit in the five diffeent categories.  Note the 4 second intervals between each pull from reddit.
top_subreddit = subreddit.top(limit=LIMIT);


time.sleep(4)
hot_subreddit = subreddit.hot(limit=LIMIT);


time.sleep(4)
new_subreddit = subreddit.new(limit=LIMIT);


time.sleep(4)
controversial_subreddit = subreddit.controversial(limit=LIMIT);


# time.sleep(4)
# gilded_subreddit = subreddit.controversial(limit=LIMIT);



lst = ['top_subreddit', 'hot_subreddit', 'new_subreddit', 'controversial_subreddit'] #, 'gilded_subreddit']
# lst = ['top_subreddit', 'hot_subreddit']

data_dict = {
    'title'    : [],
    'score'    : [],
    'id'       : [],
    'url'      : [],
    'comms_num': [],
    'created'  : [],
    'body'     : [],
    'category' : [],
    'class'    : []
}

duplicates_dict = {
    'title'    : [],
    'score'    : [],
    'id'       : [],
    'url'      : [],
    'comms_num': [],
    'created'  : [],
    'body'     : [],
    'category' : [],
    'class'    : []
}


for item in lst:
    # print(f'category for Lego Star Wars: {item}')
    print(f'category for {SUBREDDIT}: {item}')
    for submission in eval(item):
#         print(submission, submission.id)
        if submission.id in set(data_dict['id']):
            duplicates_dict['title'].append(submission.title)
            duplicates_dict['score'].append(submission.score)
            duplicates_dict['id'].append(submission.id)
            duplicates_dict['url'].append(submission.url)
            duplicates_dict['comms_num'].append(submission.num_comments)
            duplicates_dict['created'].append(submission.created)
            duplicates_dict['body'].append(submission.selftext)
            duplicates_dict['category'].append(item)
            duplicates_dict['class'] = 1
        
        else:
#             print(submission)
            data_dict['title'].append(submission.title)
            data_dict['score'].append(submission.score)
            data_dict['id'].append(submission.id)
            data_dict['url'].append(submission.url)
            data_dict['comms_num'].append(submission.num_comments)
            data_dict['created'].append(submission.created)
            data_dict['body'].append(submission.selftext)
            data_dict['category'].append(item)
            data_dict['class'] = 1
#             print(item)
# print(len(data_dict['id']))
            


# constructing a data frame from the data
# positive_data = pd.DataFrame(data_dict)
negative_data = pd.DataFrame(data_dict)


# This function is taken from Rodrigues' website.  It converts the initial time stamp, which is a UNIX timestamp,
# into something we can understand.
def get_date(created):
    return dt.datetime.fromtimestamp(created)


# applying the timestamp function to create a new row.  This is how Rodrigues did it.  We follow his lead.
_timestamp = negative_data['created'].apply(get_date)


# Now, we create a new row in the dataframe to hold the new time stamp information
negative_data = negative_data.assign(timestamp = _timestamp)


print()
print('The totle number of posts pulled from the surver:' + color.BOLD + f'  {len(data_dict["id"]) + len(duplicates_dict["id"])}' + color.END)
print()
print('The number of unique posts in the new dictionary:' + color.BOLD + f'  {len(data_dict["id"])}' + color.END)
print('The number of posts in the duplicates dictionary:' + color.BOLD + f'  {len(duplicates_dict["id"])}' + color.END)
print()
print(f'Our new dataframe is defined as: positive_data')



end_time = round(time.time() - start_time, 3)
print(f'time: {end_time} seconds')
end_time_minutes = int(end_time/ 60)
end_time_seconds = round(end_time % 60, 3)

print(f'time: {end_time_minutes} minutes, {end_time_seconds} seconds')
      
      

category for democrats: top_subreddit


OAuthException: invalid_grant error processing request

In [30]:
Negative_data_df = negative_data[negative_data['body'] != '']['body'].reset_index()
print('There are' + color.BOLD + f' {Negative_data_df.shape[0]} ' + color.END + 'Democrat posts that have content in the body of the posts.')



There are 208 Democrat posts that have content in the body of the posts.


In [16]:
#### ONLY RUN THIS CELL IF YOU WANT TO OVERRIDE THE CURRENT DATA, WHICH HAS 200 POSTS


# Negative_data_df = negative_data[negative_data['body'] != '']['body'].reset_index().iloc[:200]
# Negative_data_df.drop(columns=['index'], inplace=True)
# Negative_data_df.to_csv('./data/democrats.csv', index=False)

#### Define the Democrats (negative/0) dataframe

In [3]:
negative_data_df = pd.read_csv('./data/democrats.csv')
negative_data_df

,body
0,"I know it all Kamala and Walz, but just a litt..."
1,I agree with Hamill here!
2,I’m continually stunned that his cult base has...
3,It’s confirmed! I can’t wait for her to Win
4,I just need to rant and apologize to everyone ...
...,...
195,Our party has nominated the only Catholic Pres...
196,My reply to a Maga friend name calling of Harr...
197,Of the clowns being put in charge which is the...
198,She says she will be voting for Trump. She’s a...


#### Create the combined text dataframe

In [4]:
# combined text data

# create an Account variable
positive_data_df['Account'] = 'Republicans'
negative_data_df['Account'] = 'Democrats'

# combine the two dataframes into one
Text_df = pd.concat([negative_data_df, positive_data_df], ignore_index=True)

# rename the column 'body' as 'Text'
Text_df.rename(columns={'body':'Text'}, inplace=True)
Text_df.to_csv('./data/text_data.csv', index=False)

In [5]:
text_df = pd.read_csv('./data/text_data.csv')
text_df

,Text,Account
0,"I know it all Kamala and Walz, but just a litt...",Democrats
1,I agree with Hamill here!,Democrats
2,I’m continually stunned that his cult base has...,Democrats
3,It’s confirmed! I can’t wait for her to Win,Democrats
4,I just need to rant and apologize to everyone ...,Democrats
...,...,...
395,In recent months much has been discussed about...,Republicans
396,It is my wish that they would have the respect...,Republicans
397,\nhttps://youtu.be/BfK0ynBtVoQ?si=DKJP4_qaHevU...,Republicans
398,My biggest problem with voting for Romney is h...,Republicans


### Preparing the Data with Feature Transformations

We will transform the data to a matrix of numbers for sci-kit learn and Apache Spark.  We will use the Bag of Words technique to accomplish this.  In this method each text field is transformed so that it is represented by a number of columns, each of which represent a word count associated to each word in the text.

In [6]:
# encoding the `Account` variable: "0" of Democrats and "1" for Republicans 
text_df.Account = text_df.Account.map({"Democrats":0, "Republicans":1})
text_df

,Text,Account
0,"I know it all Kamala and Walz, but just a litt...",0
1,I agree with Hamill here!,0
2,I’m continually stunned that his cult base has...,0
3,It’s confirmed! I can’t wait for her to Win,0
4,I just need to rant and apologize to everyone ...,0
...,...,...
395,In recent months much has been discussed about...,1
396,It is my wish that they would have the respect...,1
397,\nhttps://youtu.be/BfK0ynBtVoQ?si=DKJP4_qaHevU...,1
398,My biggest problem with voting for Romney is h...,1


#### Stop Words

Create a set of common words in English to remove from the analysis.

In [15]:
# stop words

from nltk.corpus import stopwords

# Create a set of stop words 
STOP_WORDS = set(stopwords.words('english'))

stop_words = set()
for word in STOP_WORDS:
    new_word = str()                         # create a string without punctuation
    for char in word:                        # look at each character in the word
        if char.isalpha():                   # determine if the character is alphabetic
            new_word += char                 # if it is alphabetic, add it to the new word
    if new_word != '':
        stop_words.add(new_word)

# stop_words

#### Feature Function

Use to transform a text instance into a list of individual words with counts.  

In [22]:
def FeatureFunction(review: str) -> list[tuple[str, int]]:

    import numpy as np
    from nltk.stem import WordNetLemmatizer

    # instantiate important data types for storing processed data
    review_array = np.array(review.strip().lower().split())  # list of potential words from string
    dict_counts = dict()                                     # dictionary for holding word counts

    # Initialize wordnet lemmatizer
    wnl = WordNetLemmatizer()
    
    # consider each word, determine if any of its characters are not alphabetic
    for word in review_array:

        new_word = str()                                     # create a string without punctuation
        for char in word:                                    # look at each character in the word
            if char.isalpha():                               # determine if the character is alphabetic
                new_word += char                             # if it is alphabetic, add it to the new word
        if new_word != '':                                   # make sure the new string is not empty

            newer_word = wnl.lemmatize(new_word, pos="v")    # lemetize the word
            if newer_word not in stop_words:                 # check that the new word is not a common word listed in the stop words
            
                # update the word count for the new word
                if newer_word in dict_counts:                # determine if the new_word has been counted
                    dict_counts[newer_word] += 1
                else:                                        # add new words to the counts dictionary
                    dict_counts[newer_word] = 1
                
    output = list()                                          # list of (word, count) tuples
    for word in dict_counts:
        output.append((word, dict_counts[word]))
        
    return output

#### Example using the Feature Function 

In [7]:
review = "My bounty is as boundless as the sea My love as deep the more I give to thee The more I have for both are infinite"

In [8]:
bnaivebayes.FeatureFunction(review)

[('bounty', 1),
 ('boundless', 1),
 ('sea', 1),
 ('love', 1),
 ('deep', 1),
 ('give', 1),
 ('thee', 1),
 ('infinite', 1)]

#### Score Function

Use to determine which label should be assigned to a particular text instance

In [25]:
def Score(review: str, weights: dict[str, list[float]]) -> list[float]:
    ''' Calculate the positive and negative score of the given review given the weights '''

    # create the bag of words associated with the review instance
    bag_of_words = FeatureFunction(review)

    # found the positive class and negative class scores
    positive = int()
    negative = int()
    for word, count in bag_of_words:

        try:
            positive += weights[word][0]*count
            negative += weights[word][1]*count
        except:
            pass

    # create the output list
    output = list()
    output.append(positive)
    output.append(negative)
    
    return output

#### Example using the Score function

In [9]:
review = "The example text for the demonstration"

In [10]:
weights = {"the": [2, 0], "example": [0, -3], "text": [0, 1], "for": [0, 0], "demonstration": [2, 0]}

In [11]:
bnaivebayes.Score(review, weights)

[2, -2]

#### Naive Bayes algorithm

Train a Naive Bayes algorithm by taking in positive and negative reviews and creating a relative frequency count for each class label from the words present in the reviews.

In [29]:
def check_string(text: str):
    return text.isalpha()

def NaiveBayes(pos_reviews: list[str],
               neg_reviews: list[str]) -> dict[str, list[float]]:

    ''' Take positive and negative reviews and return a dictionary of weights for each of the unique words in the vocabulary '''
    
    # create weights dictionary
    naive_weights = dict()

    # Initialize wordnet lemmatizer
    wnl = WordNetLemmatizer()
    
    num_positive_words = 0
    for review in pos_reviews:
        review_list = review.strip().lower().split()
        
        for word in review_list:

            new_word = str()                                   # create a string without punctuation
            for char in word:                                  # look at each character in the word
                if check_string(char):                         # determine if the character is alphabetic
                    new_word += char                           # if it is alphabetic, add it to the new word
            if new_word != '':                                 # make sure the new string is not empty

                newer_word = wnl.lemmatize(new_word, pos="v")  # lemetize the word
                if newer_word not in stop_words:               # determine if the word is a stop word
            
                    num_positive_words += 1
                    if newer_word in naive_weights:
                        naive_weights[newer_word][0] += 1
                    else:
                        naive_weights[newer_word] = [1, 0]
    
    num_negative_words = 0
    for review in neg_reviews:
        review_list = review.strip().lower().split()
        
        for word in review_list:

            new_word = str()                                    # create a string without punctuation
            for char in word:                                   # look at each character in the word
                if check_string(char):                          # determine if the character is alphabetic
                    new_word += char                            # if it is alphabetic, add it to the new word
            if new_word != '':                                  # make sure the new string is not empty
                
                newer_word = wnl.lemmatize(new_word, pos="v")   # lemetize the word
                if newer_word not in stop_words:                # determine if the word is a stop word

                    num_negative_words += 1
                    if newer_word in naive_weights:
                        naive_weights[newer_word][1] += 1
                    else:
                        naive_weights[newer_word] = [0, 1]
 
            
    for item in naive_weights:
        naive_weights[item] = [naive_weights[item][0] / num_positive_words, naive_weights[item][1] / num_negative_words]
            
    return naive_weights

#### Naive Bayes example

Take in a list of positive and negative reviews and determine the relative frequency of word in the two classes.

In [12]:
pos_reviews = [
        "I liked the film. Some of the action scenes were very interesting, tense and well done. I especially liked the opening scene which had a semi truck in it. A very tense action scene that seemed well done",
        "I enjoyed The Night Listener very much. It's one of the better movies of the summer. Robin Williams gives one of his best performances. In fact, the entire cast was very good",
        "I absolutely LOVED this film! I do not at all relate to all the other comments I have read about it. I was COMPLETELY enthralled through every second! I found the story gripping, the acting intense, and the direction spot-on"
    ]

In [13]:
neg_reviews = [
        "The film is bad. There is no other way to say it. The story is weak and outdated, especially for this country. I don't think most people know what a 'walker' is or will really care. I felt as if I was watching a movie from the 70's.",
        "I saw this movie at a drive-in in 1959. Until 'Howard the Duck' I considered this the worst movie I had ever seen. This movie tried to combine all the genera in one; comedy, horror, teenage angst, and the hot rod that must have sired My Mother, The Car.",
        "I rented this on DVD and I kind of feel bad since Dawson and Lugacy are so earnest about it in the DVD comments. It's not a bad movie exactly, but it's one of those films that desperately wants to be a deep comment on human nature while not realizing that its story is practically a genre"
    ]

In [14]:
example_weights = bnaivebayes.NaiveBayes(pos_reviews, neg_reviews)
example_weights

{'like': [0.0392156862745098, 0.0],
 'film': [0.0392156862745098, 0.028985507246376812],
 'action': [0.0392156862745098, 0.0],
 'scenes': [0.0196078431372549, 0.0],
 'interest': [0.0196078431372549, 0.0],
 'tense': [0.0392156862745098, 0.0],
 'especially': [0.0196078431372549, 0.014492753623188406],
 'open': [0.0196078431372549, 0.0],
 'scene': [0.0392156862745098, 0.0],
 'semi': [0.0196078431372549, 0.0],
 'truck': [0.0196078431372549, 0.0],
 'seem': [0.0196078431372549, 0.0],
 'enjoy': [0.0196078431372549, 0.0],
 'night': [0.0196078431372549, 0.0],
 'listener': [0.0196078431372549, 0.0],
 'much': [0.0196078431372549, 0.0],
 'one': [0.0392156862745098, 0.028985507246376812],
 'better': [0.0196078431372549, 0.0],
 'movies': [0.0196078431372549, 0.0],
 'summer': [0.0196078431372549, 0.0],
 'robin': [0.0196078431372549, 0.0],
 'williams': [0.0196078431372549, 0.0],
 'give': [0.0196078431372549, 0.0],
 'best': [0.0196078431372549, 0.0],
 'performances': [0.0196078431372549, 0.0],
 'fact':

#### Transform the weights using the Natural Logarithm

In [19]:
import math

def LogWeights(nb_weights: dict[str, list[float]]) -> None:
    ''' Apply the natural logarthm to each non-zero entry in the weights dictionary,
    setting zero entries to -10000 '''
    
    for item in nb_weights:
        nb_weights[item] = [math.log(num) if num != 0 else -10000 for num in nb_weights[item]]


#### Example using the Logarithm Transformation

In [15]:
bnaivebayes.LogWeights(example_weights)
example_weights

{'like': [-3.2386784521643803, -10000],
 'film': [-3.2386784521643803, -3.5409593240373143],
 'action': [-3.2386784521643803, -10000],
 'scenes': [-3.9318256327243257, -10000],
 'interest': [-3.9318256327243257, -10000],
 'tense': [-3.2386784521643803, -10000],
 'especially': [-3.9318256327243257, -4.23410650459726],
 'open': [-3.9318256327243257, -10000],
 'scene': [-3.2386784521643803, -10000],
 'semi': [-3.9318256327243257, -10000],
 'truck': [-3.9318256327243257, -10000],
 'seem': [-3.9318256327243257, -10000],
 'enjoy': [-3.9318256327243257, -10000],
 'night': [-3.9318256327243257, -10000],
 'listener': [-3.9318256327243257, -10000],
 'much': [-3.9318256327243257, -10000],
 'one': [-3.2386784521643803, -3.5409593240373143],
 'better': [-3.9318256327243257, -10000],
 'movies': [-3.9318256327243257, -10000],
 'summer': [-3.9318256327243257, -10000],
 'robin': [-3.9318256327243257, -10000],
 'williams': [-3.9318256327243257, -10000],
 'give': [-3.9318256327243257, -10000],
 'best': [

#### Prediction Accuracy

Use this to determine the accuracy of the Naive Bayes predictions.

In [34]:
def PredAccuracy(pos_reviews: list[str],
                 neg_reviews: list[str],
                 weights: dict[str, list[float]]) -> float:

    ''' Determine the prediction accuracy of the given reviews using the indicated weights '''
    
    num_correct = int()
    
    for review in pos_reviews:
        review_scores = Score(review, weights)
        
        if review_scores[0] > review_scores[1]:
            num_correct += 1
            
    for review in neg_reviews:
        review_scores = Score(review, weights)
        
        if review_scores[0] < review_scores[1]:
            num_correct += 1
            
    prediction_accuracy = num_correct / len(pos_reviews + neg_reviews)
        
    
    return prediction_accuracy
    # return num_correct

#### Example calculating the prediction accuracy

Here the positive_reviews and negative_reviews and example_weights from the preceding examples is being used.

In [16]:
bnaivebayes.PredAccuracy(pos_reviews, neg_reviews, example_weights)

1.0

#### Split the data, creating a Training and a Testing set.

In [30]:
positive = pd.read_csv('./data/republicans.csv')
negative = pd.read_csv('./data/democrats.csv')

In [31]:
pos_list = list(positive.body)
neg_list = list(negative.body)

In [32]:
len(pos_list)

200

In [103]:
positive_shuffled = positive.sample(frac=1, random_state=42)
pos_list_train = list(positive_shuffled['body'].iloc[:180])
pos_list_validation = list(positive_shuffled['body'].iloc[180:190])
pos_list_test = list(positive_shuffled['body'].iloc[190:])

In [104]:
print(len(pos_list_train))
print(len(pos_list_validation))
print(len(pos_list_test))

180
10
10


In [105]:
len(neg_list)

200

In [106]:
negative_shuffled = negative.sample(frac=1, random_state=42)
neg_list_train = list(negative_shuffled['body'].iloc[:180])
neg_list_validation = list(negative_shuffled['body'].iloc[180:190])
neg_list_test = list(negative_shuffled['body'].iloc[190:])

In [107]:
print(len(neg_list_train))
print(len(neg_list_validation))
print(len(neg_list_test))

180
10
10


In [108]:
len(pos_list_train + neg_list_train)

360

In [109]:
nb_weights = bnaivebayes.NaiveBayes(pos_list_train, neg_list_train)
bnaivebayes.LogWeights(nb_weights)

In [110]:
# training accuracy
bnaivebayes.PredAccuracy(pos_list_train, neg_list_train, nb_weights)

0.9555555555555556

In [111]:
# validation accuracy
bnaivebayes.PredAccuracy(pos_list_validation, neg_list_validation, nb_weights)

0.5

In [112]:
bnaivebayes.PredAccuracy(pos_list_test, neg_list_test, nb_weights)

0.65

# !!!!!WAY OVERFITTING!!!!!

We believe the primary issue is bad and unrepresentative data.  We will work to get better data.

#### Exploring the incorrectly classified cases.

In [113]:
total_number_of_testing_cases = len(pos_list_validation) + len(neg_list_validation)
total_number_of_testing_cases

20

In [114]:
number_of_cases = len(pos_list_validation) + len(neg_list_validation)
number_missclassified = number_of_cases*(1 - bnaivebayes.PredAccuracy(pos_list_validation, neg_list_validation, nb_weights))
round(number_missclassified)

10

In [115]:
if bnaivebayes.Score(pos_list_validation[0], nb_weights)[0] > bnaivebayes.Score(pos_list_validation[0], nb_weights)[1]:
    print('positive case')
else:
    print('negative case')

positive case


In [116]:
# locate the first positive case that is missclassified
i = 0
while bnaivebayes.Score(pos_list_validation[i], nb_weights)[0] > bnaivebayes.Score(pos_list_validation[i], nb_weights)[1]:
    i += 1
i

7

In [117]:
# the naive bayes score of the missclassified case with index 7
bnaivebayes.Score(pos_list_validation[7], nb_weights)

[-10000, -8.936955604225226]

In [118]:
# the missclassified case
pos_list_validation[7]

'This is just depressing\n\n\n'

In [119]:
# the feature function of this test case
bnaivebayes.FeatureFunction(pos_list_validation[7])

[('depress', 1)]

In [262]:
new_word = str()
for char in "it.":
    if char.isalpha():
        new_word += char
new_word

'it'

In [253]:
new_word = str()
for char in "bad.":
    if char.isalpha():
        new_word += char
new_word

'bad'

In [20]:
## IMPORT THE NLTK LIBRARY
from nltk.tokenize import word_tokenize

def tokenizeReview(reviews: list[str]) -> list[list[str]]:
    ''' Create a list of tokens for each review and return a list of the token lists '''
    
    tokenized_reviews = list()
    for review in reviews:
        tokenized_reviews.append(word_tokenize(review))
        
    return tokenized_reviews

In [21]:
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
# nltk.download('punkt')

def cleanTokenizedReview(reviews: list[str]) -> list[list[str]]:

    clean_tokenized_reviews = list()
    punctuation = list(string.punctuation)
    stop_words = list(stopwords.words('english'))
    
    for review in reviews:
        clean_review = list()
        for word in review:

            new_word = str()                         # create a string without punctuation
            for char in word:                        # look at each character in the word
                if char.isalnum():                   # determine if the character is alpha-numeric
                    new_word += char                 # if it is alphabetic, add it to the new word

            if new_word != '':
                if new_word not in punctuation:
                    new_word = new_word.lower()
                    if new_word not in stop_words:
                        clean_review.append(new_word)

        clean_tokenized_reviews.append(clean_review)

        #     if word not in punctuation:
        #         new_word = word.lower()
        #         if new_word not in stop_words:
        #             clean_review.append(new_word)
        # clean_tokenized_reviews.append(clean_review)
                
        
    
    return clean_tokenized_reviews

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/blakewallace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [105]:
def FeatureFunction_list(review: list) -> dict: #list[tuple[str, int]]:

    dict_counts = dict()                         # dictionary for holding word counts

    for word in review:

        # print(word)
        new_word = str()                         # create a string without punctuation
        for char in word:                        # look at each character in the word
            if char.isalpha():                   # determine if the character is alphabetic
                new_word += char                 # if it is alphabetic, add it to the new word

        # print(new_word)
        if new_word != '':                       # make sure the new string is not empty
            
            # update the word count for the new word
            if new_word in dict_counts:          # determine if the new_word has been counted
                dict_counts[new_word] += 1
            else:                                # add new words to the counts dictionary
                dict_counts[new_word] = 1
                
    output = list()                              # list of (word, count) tuples
    for word in dict_counts:
        output.append((word, dict_counts[word]))
        
    return dict_counts

In [106]:
# modifying FeatureFunction to take in a string
def FeatureFunction(review: str) -> dict: #list[tuple[str, int]]:

    dict_counts = dict()                         # dictionary for holding word counts

    for word in review.lower().split():

        # print(word)
        new_word = str()                         # create a string without punctuation
        for char in word:                        # look at each character in the word
            if char.isalpha():                   # determine if the character is alphabetic
                new_word += char                 # if it is alphabetic, add it to the new word

        # print(new_word)
        if new_word != '':                       # make sure the new string is not empty
            
            # update the word count for the new word
            if new_word in dict_counts:          # determine if the new_word has been counted
                dict_counts[new_word] += 1
            else:                                # add new words to the counts dictionary
                dict_counts[new_word] = 1
                
    output = list()                              # list of (word, count) tuples
    for word in dict_counts:
        output.append((word, dict_counts[word]))
        
    return dict_counts

In [107]:
text_df

,Text,Account,Text_dict
0,"I know it all Kamala and Walz, but just a litt...",0,"{'know': 1, 'kamala': 1, 'walz': 1, 'little': ..."
1,I agree with Hamill here!,0,"{'agree': 1, 'hamill': 1}"
2,I’m continually stunned that his cult base has...,0,"{'continually': 1, 'stunned': 1, 'cult': 1, 'b..."
3,It’s confirmed! I can’t wait for her to Win,0,"{'confirmed': 1, 'wait': 1, 'win': 1}"
4,I just need to rant and apologize to everyone ...,0,"{'need': 1, 'rant': 1, 'apologize': 1, 'everyo..."
...,...,...,...
395,The election results are a real opportunity to...,1,"{'election': 1, 'results': 1, 'real': 1, 'oppo..."
396,I am stunned and disgusted that this administr...,1,"{'stunned': 1, 'disgusted': 1, 'administration..."
397,In recent months much has been discussed about...,1,"{'recent': 1, 'months': 1, 'much': 1, 'discuss..."
398,It is my wish that they would have the respect...,1,"{'wish': 4, 'would': 2, 'respect': 1, 'guest':..."


In [108]:
text_df['Text_dict'] = text_df.Text.map(lambda x: FeatureFunction_list(cleanTokenizedReview(tokenizeReview([x]))[0]))

In [109]:
text_df

,Text,Account,Text_dict
0,"I know it all Kamala and Walz, but just a litt...",0,"{'know': 1, 'kamala': 1, 'walz': 1, 'little': ..."
1,I agree with Hamill here!,0,"{'agree': 1, 'hamill': 1}"
2,I’m continually stunned that his cult base has...,0,"{'continually': 1, 'stunned': 1, 'cult': 1, 'b..."
3,It’s confirmed! I can’t wait for her to Win,0,"{'confirmed': 1, 'wait': 1, 'win': 1}"
4,I just need to rant and apologize to everyone ...,0,"{'need': 1, 'rant': 1, 'apologize': 1, 'everyo..."
...,...,...,...
395,The election results are a real opportunity to...,1,"{'election': 1, 'results': 1, 'real': 1, 'oppo..."
396,I am stunned and disgusted that this administr...,1,"{'stunned': 1, 'disgusted': 1, 'administration..."
397,In recent months much has been discussed about...,1,"{'recent': 1, 'months': 1, 'much': 1, 'discuss..."
398,It is my wish that they would have the respect...,1,"{'wish': 4, 'would': 2, 'respect': 1, 'guest':..."


In [110]:
textDataFrame = pd.DataFrame()

def wordCountsColumns(dictionary: dict) -> pd.DataFrame:

    new_column = list()

    for word in dictionary:
        print(word, dictionary[word])

        if word in textDataFrame.columns:
            print(f"{word} already present")
        else:
            textDataFrame.append({word:dictionary[word]}, axis=1)

In [111]:
def NaiveBayes(pos_reviews: list[str],
               neg_reviews: list[str]) -> dict[str, list[float]]:

    ''' Take positive and negative reviews and return a dictionary of weights for each of the unique words in the vocabulary '''
    
    # create weights dictionary
    naive_weights = dict()
    
    num_positive_words = 0
    for review in pos_reviews:
        review_list = review.lower().split()
        
        for word in review_list:
            
            num_positive_words += 1
            if word in naive_weights:
                naive_weights[word][0] += 1
            else:
                naive_weights[word] = [1, 0]
    
    num_negative_words = 0
    for review in neg_reviews:
        review_list = review.lower().split()
        
        for word in review_list:
            
            num_negative_words += 1
            if word in naive_weights:
                naive_weights[word][1] += 1
            else:
                naive_weights[word] = [0, 1]
 
            
    for item in naive_weights:
        naive_weights[item] = [naive_weights[item][0] / num_positive_words, naive_weights[item][1] / num_negative_words]
            
    return naive_weights
        
        
        

In [112]:
positive_list = list(positive_data_df.body)
negative_list = list(negative_data_df.body)

In [113]:
positive_array = np.array(positive_data_df.body)
negative_array = np.array(negative_data_df.body)

In [126]:
bow_weights = NaiveBayes(positive_array, negative_array)
bow_weights

{'so': [0.004443422967900099, 0.003895714713814804],
 'obviously': [3.8305370412931895e-05, 5.993407252022775e-05],
 'as': [0.004520033708725963, 0.006053341324543003],
 'the': [0.040986746341837126, 0.04237338927180102],
 'title': [0.00011491611123879568, 0.0],
 'reads,': [3.8305370412931895e-05, 0.0],
 'i': [0.016662836129625373, 0.020976925382079712],
 'am': [0.0014172987052784801, 0.001438417740485466],
 'a': [0.019957097985137517, 0.021156727599640397],
 'democrat': [0.0011491611123879567, 0.0007791429427629607],
 'and': [0.026239178732858345, 0.027030266706622716],
 "i've": [0.0005362751857810465, 0.000479472580161822],
 'been': [0.0023749329656017775, 0.001917890320647288],
 'reading': [7.661074082586379e-05, 0.000239736290080911],
 'posts': [0.0005745805561939783, 0.00029967036260113877],
 'you': [0.012257718532138206, 0.007072220557386874],
 'guys': [0.00030644296330345516, 0.0001198681450404555],
 'put': [0.0006128859266069103, 0.000719208870242733],
 'up': [0.001876963150233

In [115]:
import math

def LogWeights(nb_weights: dict[str, list[float]]) -> None:
    ''' Apply the natural logarthm to each non-zero entry in the weights dictionary,
    setting zero entries to -10000 '''
    
    for item in nb_weights:
        nb_weights[item] = [math.log(num) if num != 0 else -10000 for num in nb_weights[item]]


In [116]:
LogWeights(bow_weights)

In [124]:
# logged-weights
bow_weights

{'so': [-5.41633026084237, -5.547878121289838],
 'obviously': [-10.169920451948736, -9.722265391185474],
 'as': [-5.3992358274830705, -5.107144874344215],
 'the': [-3.194506524492783, -3.1612347252889017],
 'title': [-9.071308163280625, -10000],
 'reads,': [-10.169920451948736, -10000],
 'i': [-4.094574420860051, -3.864332236702015],
 'am': [-6.559002539304511, -6.544211560837529],
 'a': [-3.9141704101953683, -3.8557973342521774],
 'democrat': [-6.76872307028658, -7.157316033723938],
 'and': [-3.64050161368651, -3.610798051682796],
 "i've": [-7.530863122333477, -7.642823849505638],
 'been': [-6.042786066903643, -6.256529488385747],
 'reading': [-9.47677327138879, -8.335971030065584],
 'posts': [-7.461870250846525, -8.112827478751374],
 'you': [-4.401599456154963, -4.951580766719809],
 'guys': [-8.0904789102689, -9.02911821062553],
 'put': [-7.397331729708954, -7.2373587413974745],
 'up': [-6.278100153838109, -6.1387464527293645],
 'here.': [-8.560482539514634, -9.02911821062553],
 'wit

In [118]:
def Score(review: str, weights: dict[str, list[float]]) -> list[float]:
    ''' Calculate the positive and negative score of the given review given the weights '''
    
    bag_of_words = FeatureFunction(review)
    
    positive = int()
    negative = int()
    for word, count in bag_of_words.items():
        
        positive += weights[word][0]*count
        negative += weights[word][1]*count
        
    output = list()
    output.append(positive)
    output.append(negative)
    
    return output
    
def PredAccuracy(pos_reviews: list[str],
                 neg_reviews: list[str],
                 weights: dict[str, list[float]]) -> float:

    ''' Determine the prediction accuracy of the given reviews using the indicated weights '''
    
    num_correct = int()
    
    for review in pos_reviews:
        review_scores = Score(review, weights)
        
        if review_scores[0] > review_scores[1]:
            num_correct += 1
            
    for review in neg_reviews:
        review_scores = Score(review, weights)
        
        if review_scores[0] < review_scores[1]:
            num_correct += 1
            
    prediction_accuracy = num_correct / len(pos_reviews + neg_reviews)
        
    
    return prediction_accuracy

In [119]:
positive_array[0]

'So obviously as the title reads, I am a democrat and I\'ve been reading the posts you guys put up here. With the election going on I figured I would try and get a perspective from the other side. I would like to say that the vast majority of you all seem like great people. I know that Reddit leans so far left that it might fall off the edge of the internet eventually, so I understand how being republican/conservative on this app can be frustrating. I actually agree with a lot of the points and arguments you guys make, I do disagree with some, but not nearly as many as I thought I would. I just think that it\'s the far extremists on both sides that are making everything seem so damn polarized right now. In reality, I bet 95% of us want similar things, but the media makes it seem as though we are polar opposites. I\'m not sure about you all but I am tired of all of this. I am tired of the hate from the extremists on both ends, I\'m tired of the politics smothering social media platforms

In [120]:
FeatureFunction(positive_array[0])

{'so': 8,
 'obviously': 1,
 'as': 6,
 'the': 23,
 'title': 1,
 'reads': 1,
 'i': 23,
 'am': 5,
 'a': 9,
 'democrat': 1,
 'and': 5,
 'ive': 2,
 'been': 2,
 'reading': 2,
 'posts': 2,
 'you': 7,
 'guys': 2,
 'put': 1,
 'up': 1,
 'here': 2,
 'with': 3,
 'election': 1,
 'going': 1,
 'on': 5,
 'figured': 1,
 'would': 4,
 'try': 1,
 'get': 1,
 'perspective': 1,
 'from': 2,
 'other': 2,
 'side': 1,
 'like': 5,
 'to': 5,
 'say': 2,
 'that': 8,
 'vast': 1,
 'majority': 1,
 'of': 15,
 'all': 8,
 'seem': 3,
 'great': 2,
 'people': 1,
 'know': 2,
 'reddit': 2,
 'leans': 1,
 'far': 4,
 'left': 1,
 'it': 4,
 'might': 1,
 'fall': 1,
 'off': 1,
 'edge': 1,
 'internet': 1,
 'eventually': 1,
 'understand': 1,
 'how': 1,
 'being': 2,
 'republicanconservative': 1,
 'this': 3,
 'app': 1,
 'can': 1,
 'be': 1,
 'frustrating': 1,
 'actually': 1,
 'agree': 1,
 'lot': 4,
 'points': 1,
 'arguments': 1,
 'make': 1,
 'do': 1,
 'disagree': 1,
 'some': 1,
 'but': 6,
 'not': 4,
 'nearly': 1,
 'many': 1,
 'thought': 1

In [121]:
Score(positive_array[0], bow_weights)

KeyError: 'reads'

In [122]:
bow_weights["reads"]

KeyError: 'reads'

In [59]:
for word, count in bow_weights.items():
    print(word, count)

so [-5.41633026084237, -5.547878121289838]
obviously [-10.169920451948736, -9.722265391185474]
as [-5.3992358274830705, -5.107144874344215]
the [-3.194506524492783, -3.1612347252889017]
title [-9.071308163280625, -10000]
reads, [-10.169920451948736, -10000]
i [-4.094574420860051, -3.864332236702015]
am [-6.559002539304511, -6.544211560837529]
a [-3.9141704101953683, -3.8557973342521774]
democrat [-6.76872307028658, -7.157316033723938]
and [-3.64050161368651, -3.610798051682796]
i've [-7.530863122333477, -7.642823849505638]
been [-6.042786066903643, -6.256529488385747]
reading [-9.47677327138879, -8.335971030065584]
posts [-7.461870250846525, -8.112827478751374]
you [-4.401599456154963, -4.951580766719809]
guys [-8.0904789102689, -9.02911821062553]
put [-7.397331729708954, -7.2373587413974745]
up [-6.278100153838109, -6.1387464527293645]
here. [-8.560482539514634, -9.02911821062553]
with [-5.221160561570567, -5.0127351898731405]
election [-7.174188178394744, -6.726533117631483]
going [-

In [61]:
PredAccuracy(positive_list, negative_list, bow_weights)

KeyError: 'S'

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Sample documents
# documents = [
#     "This is the first document.",
#     "This document is the second document.",
#     "And this is the third one.",
#     "Is this the first document?"
# ]

documents = list(text_df.Text)

# Create a CountVectorizer object
vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the documents
matrix = vectorizer.fit_transform(documents)

# Get the vocabulary (unique words)
vocabulary = vectorizer.get_feature_names_out()

# Convert the matrix to a DataFrame for better visualization
df = pd.DataFrame(matrix.toarray(), columns=vocabulary)

# Print the DataFrame
print(df)

     00  000  001  01  011  015  02  028  038  04  ...  yumi  yzbfua0jjfk  \
0     0    0    0   0    0    0   0    0    0   0  ...     0            0   
1     0    0    0   0    0    0   0    0    0   0  ...     0            0   
2     0    0    0   0    0    0   0    0    0   0  ...     0            0   
3     0    0    0   0    0    0   0    0    0   0  ...     0            0   
4     0    0    0   0    0    0   0    0    0   0  ...     0            0   
..   ..  ...  ...  ..  ...  ...  ..  ...  ...  ..  ...   ...          ...   
395   0    0    0   0    0    0   0    0    0   0  ...     0            0   
396   0    0    0   0    0    0   0    0    0   0  ...     0            0   
397   0    0    0   0    0    0   0    0    0   0  ...     0            0   
398   0    0    0   0    0    0   0    0    0   0  ...     0            0   
399   0    0    0   0    0    0   0    0    0   0  ...     0            0   

     zak  zamzam  zax  zelenskyy  zero  zombie  zone  zoom  
0      0      

In [29]:
columns_to_remove = list()
for header in df.columns:
    if header < 'abandoned':
        columns_to_remove.append(header)
    else:
        print(header)




abandoned
abated
abc7
abcnews
abdicated
abel
abide
abiding
ability
able
aborted
abortion
abou
abr
abraham
absent
absentee
absolute
absolutely
abuse
abused
abuser
abusive
abyss
acab
academic
accept
acceptable
accepted
access
accidentally
accomplish
accomplishments
according
accosted
account
accountability
accountable
accounted
accounts
accurate
accurately
accusation
accusations
accuse
accused
accusing
ace
achieve
achieved
acknowledge
acknowledging
aclu
acquaintances
acquainted
acquire
act
actblue
acted
acting
action
actions
active
actively
activist
activity
actors
acts
actual
actually
adam
add
added
adding
additional
additionally
address
addressed
addresses
addressing
adjust
adjusted
adjustment
adl
administer
administering
administration
admirable
admirably
admired
admissions
admit
admits
admitted
admittedly
adopting
adult
adults
advanced
advances
advice
advisers
advocate
advocates
affair
affairs
affect
affected
affecting
affiliated
affiliation
affinity
affluent
afford
afghanistan
afloa

In [30]:
df.drop(columns=columns_to_remove, inplace=True)

In [31]:
pd.concat([text_df, df], axis=1)

,Text,Account,Text_dict,abandoned,abated,abc7,abcnews,abdicated,abel,abide,...,yumi,yzbfua0jjfk,zak,zamzam,zax,zelenskyy,zero,zombie,zone,zoom
0,"I know it all Kamala and Walz, but just a litt...",0,"{'know': 1, 'kamala': 1, 'walz': 1, 'little': ...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,I agree with Hamill here!,0,"{'agree': 1, 'hamill': 1}",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,I’m continually stunned that his cult base has...,0,"{'continually': 1, 'stunned': 1, 'cult': 1, 'b...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,It’s confirmed! I can’t wait for her to Win,0,"{'confirmed': 1, 'wait': 1, 'win': 1}",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,I just need to rant and apologize to everyone ...,0,"{'need': 1, 'rant': 1, 'apologize': 1, 'everyo...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,The election results are a real opportunity to...,1,"{'election': 1, 'results': 1, 'real': 1, 'oppo...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
396,I am stunned and disgusted that this administr...,1,"{'stunned': 1, 'disgusted': 1, 'administration...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
397,In recent months much has been discussed about...,1,"{'recent': 1, 'months': 1, 'much': 1, 'discuss...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
398,It is my wish that they would have the respect...,1,"{'wish': 4, 'would': 2, 'respect': 1, 'guest':...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
